# Preamble



I'm conducting research on data science and its methods. Meta-data-science, so to speak.

Here, I tried to analyze the current state of and challenges in Deep Learning. Since the results stand-alone may be a bit hard to interpret, I try to show how it compares to "classical" machine learning.

Caveat emptor: I'm not a data scientist and very unexperienced. I have no idea what I'm doing.

In [ ]:
import os
import glob
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import pandasql as pdsql # *schnüff*
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sst
from functools import reduce
import pylab
import seaborn as sns
import math
import re
color = sns.color_palette()
%matplotlib inline

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input/"]).decode("utf8"))

In [ ]:
# Read each of the file
#cvRates = pd.read_csv('../input/conversionRates.csv', encoding="ISO-8859-1")
ff = pd.read_csv('../input/freeformResponses.csv', encoding="ISO-8859-1")
mc = pd.read_csv('../input/multipleChoiceResponses.csv', encoding="ISO-8859-1")
schema = pd.read_csv('../input/schema.csv', encoding="ISO-8859-1")

## Defining what deep learning is

In [ ]:
dnnerfilt = reduce(lambda a,b: a | b, (mc["WorkMethodsFrequency" + a] == v 
       for a in ("CNNs", "GANs", "RNNs")
       for v in ("Often", "Most of the time")))
dnners = mc[dnnerfilt]
nondnn = mc[~dnnerfilt]
print((len(dnners), len(nondnn)))

The code above defines Deep Learning practicioners as people who picked "Often" or "Most of the time" in their answers for at least one of CNNs, GANs, or RNNs.

Now, I'm entirely aware that Deep Learning is a term that was made up entirely to sound cool (someone needed to get a paper published — hence Deep Belief Networks were born). It is not in any way scientifically or canonically defined. I feel like this somewhat captures current use of the term, but I'm open for input on this (or adding interactivity/dropdowns so you can pick?).

In [ ]:
# method to handle those nasty selects
def splitcounts(df,name="frac",splitchr=','):
    chalsel = df.dropna()
    flat = pd.Series((i for l in chalsel.str.split(splitchr) for i in l))
    return flat.value_counts().apply(lambda x: x/len(chalsel)).to_frame(name=name)

Let's check for unwanted by-catch:

In [ ]:
splitcounts(dnners.WorkMethodsSelect).head(10)

It seems that around half of the people I selected to be "Deep Learners" also use methods like PCA and kNN, but I think that excluding everyone who uses other methods would not leave enough data. Since CNNs came out on top, I'm content with my selection for now.

In [ ]:
dnners.filter(items=['EmploymentStatus']).apply(lambda x: x.value_counts())

Sanity check: they're all employed. That's a precondition for being shown the WorkMethods question.
(One thing I don't get about the survey is that it entirely, even for part-timers, denies the possibility to be a student. I get the feeling that part-time work is the rule, not the exception for university students. And even full-time employed university students are not unherd of. So…?)

In [ ]:
# Let's exclude answers that were not from people in the CodingWorker category.
worker_label = [
    "Employed full-time",
    "Employed part-time",
    "Independent contractor, freelancer, or self-employed"
]
codingworker = reduce(lambda a,b: a | b, (mc.EmploymentStatus == l for l in worker_label)) & \
  (mc.CodeWriter == "Yes")

mc.loc[dnnerfilt & codingworker, 'MLType'] = 'Deep'
mc.loc[~dnnerfilt & codingworker, 'MLType'] = 'Classic'
dnners = mc[dnnerfilt & codingworker] # this should not change anything…
nondnn = mc[~dnnerfilt & codingworker] # These would have probably been filtered out as nans anyway
# but better safe than sorry.
show_dlgroups = lambda df: df.MLType.fillna('Excluded').value_counts().to_frame(name='Σ')
show_dlgroups(mc)

# Challenge Analsysis

In [ ]:
dnners.filter(regex='^WorkChallenge').head()

We'll work with that.

## Challenge Selection

In [ ]:
len(dnners.WorkChallengesSelect.dropna())

Yay, only 40 did not answer. It is possible that no answer means that they have no problems in data science, but I'll be so frank and assume that they ignored the question.

In [ ]:
def compare_bar_plot_prep(col, tran=lambda x: x):
    f = splitcounts(dnners[col].apply(tran), name="DNNers")
    f = f.merge(splitcounts(nondnn[col].apply(tran), name="Others"),left_index=True, right_index=True)
    f = f.reset_index().melt(id_vars=['index'],value_vars=['DNNers','Others'])
    return f
challenge_sel = compare_bar_plot_prep('WorkChallengesSelect')

In [ ]:
plt.figure(figsize=(10,20))
sns.barplot(data=challenge_sel, y='index', x='value', hue='variable', orient='h')
plt.title('Frequency of problem mentions', fontsize=16)
plt.xlabel('Fraction', fontsize=16)
plt.ylabel('Problem', fontsize=16)
plt.show()

Now here are a few things I did not expect. (Correct me if I'm misinterpreting my results.)
 - It seems that dirty data is no less of a problem for DNN practicioners.
  (And here I thought deep learning was better at ignoring noise, at least. So what kind of dirt is the problem?)
 - State of the art is an a lot bigger limitation in Deep Learning.
 - Deep Learning results can be more easily integrated.
  (Maybe I should have expected that. The things that Deep Learning promises to solve are after all usually problems that get integrated into a technical application, not int a abusiness process.)

This entirely ignored the picked frequencies. We could now ignore those answers where the frequency was picked to be Rarely or Sometimes, but I want a deeper look.

## Frequencies

In [ ]:
# (One way to visualize the frequencies might be to use stacks. 
# But I don't want to spend a day to find out how to do stacked grouped bar plots 
#  which might be hard to interpret in the end. 
# If you know an easy way: tell me.)

In [ ]:
def freqfreq(df): return df \
    .filter(regex='^WorkChallengeFrequency') \
    .fillna('Never') \
    .apply(lambda col: col.value_counts())
freqfreq(dnners)

In [ ]:
# Here is some bllllrgh:
# I want the plots in this section to be in order of the plot before.
# Now, the plot before directly used the human-readable values from the Select field,
#  but the plots in this setion will use the column names. Fun time translating…
dnn_challenge_sel = challenge_sel[challenge_sel.variable == 'DNNers']
freqschema = schema[schema.Column.apply(lambda name: name.startswith("WorkChallengeFrequency"))].copy()
freqschema['Question'] = freqschema['Question'].apply(lambda x: x.replace(
    'At work, how often did you experience these barriers or challenges within the past year? - ', ''))
dnn_challenge_sel = dnn_challenge_sel.merge(freqschema, left_on='index', right_on='Question')
sel_order = dnn_challenge_sel.set_index('Column').index
# Seriously, why is the data set this way?

In [ ]:
dnners.filter(regex='^WorkChallengeFrequency').melt().value.value_counts().to_frame(name='Σ')

So guess what, if you have a problem rarely, you don't pick that you have it in the first place. Who else is worried that Sometime may be a little bit biased towards a lower value because of the same effect? Let's proceed with caution. 

Let's first have a look at the relative frequencies:

In [ ]:
plt.figure(figsize=(5,10))
m = lambda df: freqfreq(df).apply(lambda x: x / len(df))
relfreq = m(dnners).transpose()
orfreq = relfreq[['Rarely','Sometimes','Often','Most of the time']].reindex(sel_order) # let's ignore nevers since they go into extremes
orfreq = orfreq.drop('WorkChallengeFrequencyPass')
ax = sns.heatmap(orfreq, linewidths=.5, cmap="Blues")

I see nothing extraordinary here. Maybe the tendency that if money is a problem, it's a problem all the time.

Now, what I really want to know is whether that bar plot in the previous subsection got affected by some skew between how frequent selections are.

In [ ]:
plt.figure(figsize=(5,10))
m = lambda df: freqfreq(df).apply(lambda x: x / len(df))
relfreq = (m(dnners) - m(nondnn)).transpose()
orfreq = relfreq[['Rarely','Sometimes','Often','Most of the time']].reindex(sel_order)
scalefac = dnn_challenge_sel.set_index('Column').value
orfreq = orfreq.apply(lambda c: c / scalefac)
orfreq = orfreq.drop('WorkChallengeFrequencyPass')
ax = sns.heatmap(orfreq, linewidths=.5, cmap="coolwarm")

To interpret this plot: a strong color indicates that a large part of the change between Deep and Classic practicioners comes from people answering with a certain frequency. Since there are no rows with a strong red in one column and a strong blue in another, I think the results from the previous section can be taken to be sane.

The only other thing I want to interpret into this plot is that problems that were not part of this survey occur with higher intensity in Deep Learning (i.e. Other).

# Time use

I'm interested in what costs each group the most time. First, let's see how often that question was actually answered properly:

In [ ]:
cis = schema.set_index('Column')
txt_n = lambda n: cis.Question[n]
# careful, we don't want TimeSpentStudying
cols = [q for q in mc.filter(regex='^Time').columns if 'Total must equal 100%' in txt_n(q)]

time_answered = lambda df: df[cols].apply(lambda r: r.sum(), axis=1) == 100.0
time_answer_counts = show_dlgroups(mc[time_answered(mc)])
time_answer_counts / show_dlgroups(mc)

Most for deep learning, around ⅔ for "classic". While that's slightly odd, we didn't lose anything where it hurts (in the smaller dataset).

In [ ]:
times_means = pd.DataFrame([
    dnners[time_answered(dnners)][cols].mean(),
    nondnn[time_answered(nondnn)][cols].mean()
], index=['DNNers', 'Others'])
pl=times_means.plot(kind='bar',stacked=True, fontsize=10,figsize=(10,10)) 
pass

I don't unerstand how people find it anything to read from this kind of stacked plot. But now I know that at least the sums of the means add up to 100 (percent).

In [ ]:
times_means.transpose().plot(kind='barh')
pass

No surprises here. Model building takes a more time for Deep Learning, which fits to training being immensely expensive. Everything else is proportionally reduced. (Except the time for getting things production-ready, but I feel like that's not significant enough.)

# Dataset Size

[rickvenadata](https://www.kaggle.com/rickvenadata) analyzed [how big typical used datasets are](https://www.kaggle.com/rickvenadata/big-data-or-big-hype-part-deux). If the cant about Deep Learning being able to handle bigger big data holds true, we should see something interesting here.



In [ ]:
size_order = ['<1MB', '1MB', '10MB', '100MB', '1GB', '10GB', '100GB', '1TB',
              '10TB', '100TB', '1PB', '10PB', '100PB', '1EB', '>1EB']
plt.figure(figsize=(10,5))
work_dataset_sizes = compare_bar_plot_prep('WorkDatasetSize')
sns.barplot(data=work_dataset_sizes, order=size_order,
            y='index', x='value', hue='variable', orient='h')
            #palette=['#3274a1', '#e1812c'])
plt.title('Frequency of "Most common dataset size"', fontsize=16)
plt.xlabel('Fraction', fontsize=16)
plt.ylabel('Size', fontsize=16)
plt.show()

This result looks more significant than I would have expected. So while "classic" ML focuses on data around 1GB insize, the typical size for a DNNer is around 10GB (and 100GB is fairly common, still). I'd be so frank to say that Deep Learning datasets are around one order of magnitude bigger, but the difference may be understated because of how I separated Deep Learning practicioners.

Maybe side-by-side is easier to look at:

In [ ]:
#### I'll be stealing this from rickvenadata
def parse_size(size):
    if size.endswith("MB"):
        if size.startswith("<"):
            size = float(size.rstrip("MB").lstrip("<")) * 1e5
        else:
            size = float(size.rstrip("MB")) * 1e6
    elif size.endswith("GB"):
        size = float(size.rstrip("GB")) * 1e9
    elif size.endswith("TB"):
        size = float(size.rstrip("TB")) * 1e12
    elif size.endswith("PB"):
        size = float(size.rstrip("PB")) * 1e15
    elif size.endswith("EB"):
        if size.startswith(">"):
            size = int(size.rstrip("EB").lstrip(">")) * 1e19
        else:
            size = int(size.rstrip("EB")) * 10**18
    return float(size)
fig,axs = plt.subplots(1,2,figsize=(10,4))
sns.distplot(dnners.WorkDatasetSize.dropna().apply(lambda v: math.log(parse_size(v), 10)),
             ax=axs[0],vertical=True,kde_kws={'bw': 0.45})
plt.setp(axs[0],title='DNNers')
sns.distplot(nondnn.WorkDatasetSize.dropna().apply(lambda v: math.log(parse_size(v), 10)),
             ax=axs[1],vertical=True,kde_kws={'bw': 0.45})
plt.setp(axs[1],title='Others')
pass

# Data Sharing and Storage

I think you understood the drill, by now.

In [ ]:
re_noeg = re.compile(' ?\(e.g.[^)]*\)') # argh, those e.g. contain commata. commata in a comma separated list
storetype_freqs = compare_bar_plot_prep('WorkDataStorage',
                                        tran=lambda s: re_noeg.sub('', s) if type(s) == str else s)
plt.figure(figsize=(10,5))
sns.barplot(data=storetype_freqs, y='index', x='value', hue='variable', orient='h')
plt.title('Frequency of storage type mention', fontsize=16)
plt.xlabel('Fraction', fontsize=16)
plt.ylabel('Storage type', fontsize=16)
plt.show()

Since I don't think that  Deep Learning is done from relational databases, I guess my separation of Deep Learning practicioners isn't that good.

Unfortunately, we can't do this comparison with the free form answers since they were shuffled, but I still want to have a peek:

In [ ]:
splitcounts(ff.WorkDataStorageFreeForm.dropna().apply(lambda s: s.lower()), splitchr=' ').head(10)

Maybe I didn't want to have a peek. Next.

In [ ]:
plt.figure(figsize=(10,4))
sns.barplot(data=compare_bar_plot_prep('WorkDataSharing'), y='index', x='value', hue='variable', orient='h')
plt.title('Frequency of storage type mention', fontsize=16)
plt.xlabel('Fraction', fontsize=16)
plt.ylabel('Storage type', fontsize=16)
plt.show()

# Closure 

It may be a focused analysis of a single aspect, but I hope you found it interesting. Please tell me what you think. Are there any other aspects I could analyze like this? Any difference between Deep Learning and "classic" ML that may be interesting?

While the results look like they have significance in many cases, I'm not sure they actually do. I'd be pleased if someone could suggest a method of analyzing the significance.